# 什么时候用多核心优势
- CPU密集型任务：多进程
- IO密集型任务：多线程

In [11]:
import time 
import threading
import functools

def timer(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        start_time = time.time()
        print('start time: ', start_time)
        value = func(*args, **kwargs)
        print(time.time() - start_time)
        return value
    return inner

class ThreadWithReturn(threading.Thread):
    def __init__(self, rng):
        threading.Thread.__init__(self)
        self.result = 0
        self.rng = rng

    @timer
    def run(self):
        result = 0
        for i in range(self.rng[0],self.rng[1]):
            result += i
        self.result = result

    def get_result(self):
        return self.result

In [12]:
t1 = ThreadWithReturn((0,5_000_000_0))
t2 = ThreadWithReturn((5_000_000_0,10_000_000_0))
t1.start()
t2.start()
t1.join()
t2.join()
print(t1.get_result()+t2.get_result())

start time:  1670763125.7980416
start time:  1670763125.8099592
1.7110648155212402
1.723217248916626
4999999950000000


# 多线程开发

默认是在主线程下创建子线程

In [ ]:
import threading

def task(arg):
    pass

t = threading.Thread(target=task, args=(1,))
t.start()

### `t.start()`: 当前线程准备就绪，等待CPU调度（由操作系统决定）

In [5]:
import threading

loop = 1_000_000
number = 0

def _add(count):
    global number
    for i in range(count):
        number += 1

t = threading.Thread(target=_add, args=(loop,))
t.start()

print(number)

225780


### `t.join()`: 等待当前线程的任务执行完毕后再向下继续执行

In [7]:
import threading

loop = 1_000_000
number = 0

def _add(count):
    global number
    for i in range(count):
        number += 1

t = threading.Thread(target=_add, args=(loop,))
t.start()
t.join() # 主线程等待中
print(number)

1000000


In [8]:
import threading

loop = 1_000_000
number = 0

def _add(count):
    global number
    for i in range(count):
        number += 1

def _sub(count):
    global number
    for i in range(count):
        number -= 1

t1 = threading.Thread(target=_add, args=(loop,))
t2 = threading.Thread(target=_sub, args=(loop,))

t1.start()
t1.join() # 主线程等待中
t2.start()
t2.join() # 主线程等待中

print(number)

0


In [11]:
import threading

loop = 1_000_000
number = 0

def _add(count):
    global number
    for i in range(count):
        number += 1

def _sub(count):
    global number
    for i in range(count):
        number -= 1

t1 = threading.Thread(target=_add, args=(loop,))
t2 = threading.Thread(target=_sub, args=(loop,))

t1.start()
t2.start()
t1.join() # 主线程等待中
t2.join() # 主线程等待中

print(number)

-536072


### `t.daemon`: 守护线程，必须放在start之前
`True`: 设置为守护线程，主线程执行完毕后，子线程也自动关闭  
`False`: 设置为非守护线程，主线程等待子线程，子线程执行完毕后，主线程才结束（默认）  
注意: notebook中无论如何都会执行完子线程

In [15]:
import threading
import time

def task(arg):
    time.sleep(2)
    print('hello')

t = threading.Thread(target=task, args=(1,))
t.daemon = False # 设置为守护线程
t.start()

print('world')

world


hello


### `t.name`: 线程名称，必须放在start之前

In [18]:
import threading

def task(arg):
    name = threading.current_thread().name
    print(name)

for i in range(3):
    t = threading.Thread(target=task, args=(1,))
    t.name = f'thread-{i}'
    t.start()

thread-0
thread-1
thread-2


### 自定义线程类，直接将线程需要做的事写到run方法中

In [27]:
import threading

class customThread(threading.Thread):
    def run(self):
        print(self.name)
        print(self._args)

t = customThread(args=('test',))
t.start()

Thread-33
('test',)


# 线程安全
前面同步执行示例中出现了结果混乱的现象。可以通过手动加锁，防止情况发生。

In [33]:
import threading

lock_obj = threading.RLock() # 必须使用同一把锁
def lock_decorator(func):
    def inner(*args, **kwargs):
        lock_obj.acquire() # 申请锁
        func(*args, **kwargs)
        lock_obj.release() # 释放锁
    return inner

loop = 1_000_000
number = 0

@lock_decorator
def _add(count):
    global number
    for i in range(count):
        number += 1

@lock_decorator
def _sub(count):
    global number
    for i in range(count):
        number -= 1

t1 = threading.Thread(target=_add, args=(loop,))
t2 = threading.Thread(target=_sub, args=(loop,))

t1.start()
t2.start()
t1.join() # 主线程等待中
t2.join() # 主线程等待中

print(number)

0


有些操作默认是线程安全的（内部集成了锁），使用时无需再加锁。但要注意锁是如何加上的。如列表append，多线程会产生乱序结果。  
多看官方文档。

In [37]:
import threading

lst = []

def task():
    for i in range(1_000_000):
        lst.append(i)
    print(len(lst))

for i in range(2):
    t = threading.Thread(target=task)
    t.start()

1446203
2000000


# 线程锁
- Lock：同步锁（不支持嵌套，在调用带锁function时容易出现死锁）
- RLock：递归锁

# 死锁
嵌套调用Lock出现死锁。  
并不是说RLock就万事大吉了，因为竞争资源或者彼此通信也可能造成阻塞。比如func1先申请锁1后申请锁2，func2先申请锁2再申请锁1，就会造成卡死。